# Signatures with Key Tweaking

Key tweaking is fundamental in hierarchical deterministic (HD) Bitcoin wallets and Taproot commitments, enabling the derivation of multiple keys from a single master key and committing to additional data in public keys.

## Syntax

:::{admonition} Definition: Tweaking Scheme for Signatures
:class: definition
:label: def-tweaking-scheme

A tweaking scheme $\mathsf{TwSch}$ for a signature scheme $\mathsf{Sig}$ is a tuple $(\mathcal{T}, \mathsf{TwSK}, \mathsf{TwPK})$ where:

- $\mathcal{T}$ is a set of allowed tweaks, parameterized by the public parameters $\mathsf{params}$ output by $\mathsf{Setup}$.
- $\mathsf{TwSK}(\mathsf{sk}, t) \rightarrow \tilde{\mathsf{sk}}$ is a deterministic polynomial-time algorithm that takes a secret key $\mathsf{sk}$ and a tweak $t \in \mathcal{T}$ as input and outputs a tweaked secret key $\tilde{\mathsf{sk}}$.
- $\mathsf{TwPK}(\mathsf{pk}, t) \rightarrow \tilde{\mathsf{pk}}$ is a deterministic polynomial-time algorithm that takes a public key $\mathsf{pk}$ and a tweak $t \in \mathcal{T}$ as input and outputs a tweaked public key $\tilde{\mathsf{pk}}$.

:::

## Key Tweaking for Schnorr Signatures

:::{admonition} Definition: SchnorrTS
:class: definition
:label: def-schnorrts

The tweaking scheme $\mathsf{SchnorrTS}$ for Schnorr signatures is defined as follows:

- The tweak set is $\mathcal{T} := \mathbb{Z}_p \times \mathbb{Z}_p^*$ where $\mathbb{Z}_p^* = \mathbb{Z}_p \setminus \{0\}$.
- For $(\alpha, \beta) \in \mathcal{T}$, the secret key tweaking algorithm is:
  $$
  \mathsf{TwSK}(x, (\alpha, \beta)) := \alpha + \beta \cdot x \bmod p
  $$
  for every $x \in \mathbb{Z}_p$.
- For $(\alpha, \beta) \in \mathcal{T}$, the public key tweaking algorithm is:
  $$
  \mathsf{TwPK}(X, (\alpha, \beta)) := g^\alpha \cdot X^\beta
  $$
  for every $X \in \mathbb{G}$.

:::

:::{admonition} Definition: Schnorr Signature with Key Prefixing
:class: definition
:label: def-schnorr-kp

The Schnorr signature scheme with key prefixing $\mathsf{SchnorrSigKP}[\mathcal{G}, \mathsf{H}]$ is identical to $\mathsf{SchnorrSig}[\mathcal{G}, \mathsf{H}]$ except that the hash is computed as:
$$
c = \mathsf{H.Eval}(\kappa, (R, X, m))
$$
instead of $c = \mathsf{H.Eval}(\kappa, (R, m))$, where $X$ is the public key.

:::

## Security with Tweaked Keys

The security notion of *existential unforgeability under chosen message attack with tweaked keys (EUF-CMA-TK)* extends the standard EUF-CMA notion to capture security when an adversary can obtain signatures under tweaked keys.

Informally, the adversary is given a public key and can request signatures on messages of their choice, but now with an additional capability: for each signing query, they can specify a tweak in addition to the message.
The signing oracle will return a signature that verifies under the tweaked public key (i.e., a signature created with the correspondingly tweaked secret key).
The adversary wins by outputting $(m^*, t^*, \sigma^*)$ such that the signature verifies under the tweaked public key for $t^*$, and the pair $(m^*, t^*)$ was never queried to the signing oracle.

The formal definition of EUF-CMA-TK is left as an exercise (see Exercise 3).

:::{admonition} Theorem: SchnorrSigKP with SchnorrTS is EUF-CMA-TK secure
:class: theorem
:label: thm-schnorrkp-euf-cma-tk

Let $\mathcal{G}$ be a group generation algorithm for which the discrete logarithm problem is hard and let $\mathsf{H}$ be a hash function with output space $\mathbb{Z}_p$.
Then the Schnorr signature scheme with key prefixing $\mathsf{SchnorrSigKP}[\mathcal{G}, \mathsf{H}]$ with tweaking scheme $\mathsf{SchnorrTS}$ is EUF-CMA-TK secure in the random oracle model for $\mathsf{H}$.

:::

The proof is left as an exercise (see Exercise 5).

## Exercises

:::{admonition} Exercise 1
:class: exercise
:label: ex-correctness-tweaking

Define perfect correctness for a signature scheme with tweaking.
Specifically, modify standard signature scheme correctness to ensure that signatures created with a tweaked secret key verify correctly under the corresponding tweaked public key.

:::

:::{admonition} Solution
:class: dropdown, solution

A signature scheme $\mathsf{Sig} = (\mathsf{Setup}, \mathsf{KeyGen}, \mathsf{Sign}, \mathsf{Verify})$ with tweaking scheme $\mathsf{TwSch} = (\mathcal{T}, \mathsf{TwSK}, \mathsf{TwPK})$ is *perfectly correct with tweaking* if:

1. The signature scheme $\mathsf{Sig}$ is correct (as per the standard definition).
2. For all $\lambda \in \mathbb{N}$, all tweaks $t \in \mathcal{T}$, and all messages $m \in \mathcal{M}$:
   $$
   \Pr\left[\mathsf{Verify}(\mathsf{params}, \mathsf{TwPK}(\mathsf{pk}, t), m, \sigma) = 1 \;\middle|\; \begin{array}{l}
     \mathsf{params} \leftarrow \mathsf{Setup}(1^\lambda) \\
     (\mathsf{sk}, \mathsf{pk}) \leftarrow \mathsf{KeyGen}(\mathsf{params}) \\
     \sigma \leftarrow \mathsf{Sign}(\mathsf{params}, \mathsf{TwSK}(\mathsf{sk}, t), m)
   \end{array}\right] = 1
   $$

In other words, signatures created with a tweaked secret key must always verify correctly under the corresponding tweaked public key.

:::

:::{admonition} Exercise 2 (Optional)
:class: exercise
:label: ex-schnorrts-correctness

Argue that the tweaking scheme $\mathsf{SchnorrTS}$ (see [Definition: SchnorrTS](#def-schnorrts)) is correct with the Schnorr signature scheme.
That is, show that signatures created with a tweaked secret key verify correctly under the corresponding tweaked public key.

:::

:::{admonition} Solution
:class: dropdown, solution

For SchnorrTS with Schnorr signatures, if $\mathsf{sk} = x$ and $\mathsf{pk} = X = g^x$, then for tweak $(\alpha, \beta) \in \mathcal{T}$:

- Tweaked secret key: $\tilde{x} = \mathsf{TwSK}(x, (\alpha, \beta)) = \alpha + \beta \cdot x$
- Tweaked public key: $\tilde{X} = \mathsf{TwPK}(X, (\alpha, \beta)) = g^\alpha \cdot X^\beta = g^\alpha \cdot g^{\beta x} = g^{\alpha + \beta x} = g^{\tilde{x}}$

The key observation is that the discrete logarithm relationship is preserved: $\tilde{X} = g^{\tilde{x}}$.
This means that $(\tilde{x}, \tilde{X})$ is a valid key pair that could have been output by $\mathsf{KeyGen}$.
By the correctness of Schnorr signatures, signatures created with $\tilde{x}$ will verify under $\tilde{X}$.

:::

:::{admonition} Exercise 3
:class: exercise
:label: ex-euf-cma-tk-def

Formalize the EUF-CMA-TK security notion by adapting EUF-CMA security.
Your definition should capture the informal description given in the Security with Tweaked Keys section.

:::

:::{admonition} Solution
:class: dropdown, solution

A signature scheme $\mathsf{Sig} = (\mathsf{Setup}, \mathsf{KeyGen}, \mathsf{Sign}, \mathsf{Verify})$ with tweaking scheme $\mathsf{TwSch} = (\mathcal{T}, \mathsf{TwSK}, \mathsf{TwPK})$ is *existentially unforgeable under chosen message attack with tweaking (EUF-CMA-TK)* if for all PPT adversaries $\mathcal{A}$:
$$
\mathsf{Adv}_{\mathcal{A}, \mathsf{Sig}, \mathsf{TwSch}}^{\text{EUF-CMA-TK}} := \Pr[\mathsf{Game\ EUF\text{-}CMA\text{-}TK}_{\mathsf{Sig}, \mathsf{TwSch}} = 1] = \mathsf{negl}
$$
where $\mathsf{Game\ EUF\text{-}CMA\text{-}TK}_{\mathsf{Sig}, \mathsf{TwSch}}$ is defined as follows:

**Game EUF-CMA-TK:**
```
params ← Setup(1^λ)
(sk, pk) ← KeyGen(params)
Q := ∅
(m*, t*, σ*) ← A^Sign(params, pk)
return (m*, t*) ∉ Q ∧ Verify(params, TwPK(pk, t*), m*, σ*) = 1
```

**Oracle Sign(m, t):**
```
sk_tilde ← TwSK(sk, t)
σ ← Sign(params, sk_tilde, m)
Q := Q ∪ {(m, t)}
return σ
```

Changes from the standard EUF-CMA game:
- The signing oracle now takes a tweak $t$ in addition to the message $m$
- The oracle uses the tweaked secret key $\mathsf{TwSK}(\mathsf{sk}, t)$ to sign
- The query set $\mathcal{Q}$ stores pairs $(m, t)$ instead of just messages
- The adversary outputs $(m^*, t^*, \sigma^*)$ instead of just $(m^*, \sigma^*)$
- Verification checks the signature under the tweaked public key $\mathsf{TwPK}(\mathsf{pk}, t^*)$

:::

:::{admonition} Exercise 4
:class: exercise
:label: ex-schnorrsig-euf-cma-tk-attack

Give a PPT adversary that breaks the EUF-CMA-TK security of $\mathsf{SchnorrSig}$ with the $\mathsf{SchnorrTS}$ tweaking scheme. Hints:

- $\mathsf{SchnorrSig}$ does not use key prefixing.
- You only need to use the signing oracle once.
- For simplicity, use tweaks $t_0 = (0, 1)$ and $t_1 = (1, 1)$.
- Look at the Schnorr verification equation for a signature $(R, s)$ under public key $\mathsf{TwPK}(X, t_0)$. Compute $s'$ by adding a value to $s$ such that $(R, s')$ is a valid signature for $\mathsf{TwPK}(X, t_1)$.

:::

:::{admonition} Solution
:class: dropdown, solution

The adversary $\mathcal{A}$ that breaks EUF-CMA-TK security:

**Adversary A^Sign(params, X):**
```
m ← {0,1}^*
t_0 := (0, 1)
(R, s) ← Sign(m, t_0)
c := H(R, m)
s' := s + c
t* := (1, 1)
return (m, t*, (R, s'))
```

**Why this attack succeeds:**

When querying the signing oracle with $t_0 = (0, 1)$:
- The tweaked public key is $\mathsf{TwPK}(X, (0, 1)) = g^0 \cdot X^1 = X$ (the original key).
- The signature $(R, s)$ satisfies: $g^s = R \cdot X^c$ where $c = \mathsf{H}(R, m)$.

For the forgery with $t^* = (1, 1)$:
- The tweaked public key is $\mathsf{TwPK}(X, (1, 1)) = g^1 \cdot X = g \cdot X$.
- The game verifies: $g^{s'} = g^{s + c} \stackrel{?}{=} R \cdot \mathsf{TwPK}(X, (1, 1))^c$.

The verification succeeds because:
$$
\begin{align*}
g^{s + c} &= g^s \cdot g^c \\
&= R \cdot X^c \cdot g^c \\
&= R \cdot (X \cdot g)^c \\
&= R \cdot \mathsf{TwPK}(X, (1, 1))^c
\end{align*}
$$

Since $(m, (0, 1)) \neq (m, (1, 1))$, this is a valid forgery that breaks EUF-CMA-TK security.

:::

:::{admonition} Exercise 5
:class: exercise
:label: ex-schnorr-euf-cma-proof

Explain why we cannot directly apply the EUF-CMA security proof of $\mathsf{SchnorrSig}$ to prove EUF-CMA-TK security for $\mathsf{SchnorrSig}$ with SchnorrTS tweaking.

Hint: Examine what the EUF-CMA security proof guarantees about the reduction's output and why this guarantee holds. Then consider whether a similar guarantee could be established for a reduction that uses an adversary breaking EUF-CMA-TK for $\mathsf{SchnorrSig}$ with SchnorrTS. Consider specifically the adversary from Exercise 4.

:::

:::{admonition} Solution
:class: dropdown, solution

The EUF-CMA security proof for $\mathsf{SchnorrSig}$ requires that when the reduction algorithm outputs $(z, \mathit{aux})$ with $z = (R^*, m^*)$ and $\mathit{aux} = s^*$, we have:
$$
g^{s^*} = R^* \cdot X^{\mathsf{H}(R^*, m^*)}
$$

The proof of this property relies on showing that the programmed oracle value $T[R^*, m^*]$ equals the external oracle $\mathsf{H}(R^*, m^*)$.
This is proven by contradiction: if they differ, then $T[R^*, m^*]$ must have been set during a signing query, which means $m^* \in \mathcal{Q}$.
But the reduction asserts $m^* \notin \mathcal{Q}$, giving us the contradiction.

In the EUF-CMA-TK setting, this proof breaks down:

- The adversary outputs $(m^*, t^*, \sigma^*)$ for some tweak $t^*$.
- The winning condition checks $(m^*, t^*) \notin \mathcal{Q}$, not $m^* \notin \mathcal{Q}$.
- This means the adversary could have queried the signing oracle with $m^*$ with a different tweak $t' \neq t^*$.
- Consider the adversary from Exercise 4: it queries the signing oracle with $(m, t_0 = (0,1))$ and forges on $(m, t^* = (1,1))$, reusing the same $R$ from the signature.
- When such an adversary queries $m$ with tweak $t'$, the reduction sets $T[R, m]$ to a random value. If the adversary's forgery reuses this $R$ with a different tweak $t^*$, then $T[R^*, m^*]$ has already been assigned, and it is unequal to $\mathsf{H}(R^*, m^*)$ with overwhelming probability.

Since $\mathsf{H}_{\mathcal{C}}(R^*, m^*) \neq \mathsf{H}(R^*, m^*)$ (with overwhelming probability), we cannot use the forking lemma's guarantee that $\mathsf{H}(R^*, m^*) \neq \mathsf{H}'(R^*, m^*)$ to extract the secret key from the two signatures.

:::

:::{admonition} Exercise 6
:class: exercise
:label: ex-schnorrkp-to-single

Prove the following lemma, which is an adaptation of the standard Schnorr security lemma for the EUF-CMA-TK security of $\mathsf{SchnorrSigKP}$ with SchnorrTS.

**Lemma:** Let $\mathcal{A}$ be an adversary against the EUF-CMA-TK security of $\mathsf{SchnorrSigKP}$ with $\mathsf{SchnorrTS}[\mathcal{G}, \mathsf{H}]$ in the random oracle model for $\mathsf{H}$ making at most $q_s$ queries to $\mathsf{Sign}$ and at most $q_h$ queries to $\mathsf{H}$.

Let $\mathsf{InpGen}$ be the algorithm which on input $1^\lambda$ runs $(\mathbb{G}, p, g) \leftarrow \mathcal{G}(1^\lambda)$, draws $X \leftarrow \mathbb{G}$, and returns $((\mathbb{G}, p, g), X)$.

There exists algorithm $\mathcal{C}$ that takes as input $(\mathsf{gparam}, X) \leftarrow \mathsf{InpGen}(1^\lambda)$, has access to a random oracle $\mathsf{H}$, makes at most $q_h + 1$ queries to $\mathsf{H}$, and satisfies
$$
\left|\mathsf{Adv}_{\mathcal{C}, \mathsf{InpGen}}^{\text{Single}} - \mathsf{Adv}_{\mathcal{A}, \mathsf{SchnorrSigKP}, \mathsf{SchnorrTS}}^{\text{EUF-CMA-TK}}\right| \leq \frac{q_s(q_s + q_h)}{2^{\lambda-1}}
$$
with $\mathsf{Adv}_{\mathcal{C}, \mathsf{InpGen}}^{\text{Single}}$ as defined in the forking lemma.

Moreover, when $\mathcal{C}$ succeeds (returns $(z, \mathit{aux})$ for some $(z, \mathit{aux})$), then $z = (R^*, \mathsf{TwPK}(X, t^*), m^*, t^*)$, $\mathit{aux} = s^*$, and
$$
g^{s^*} = R^* \cdot \mathsf{TwPK}(X, t^*)^{\mathsf{H}(R^*, \mathsf{TwPK}(X, t^*), m^*)}
$$

Adapt the proof of the standard Schnorr security lemma to prove this lemma.

*Note:* Applying the forking lemma to $\mathcal{C}$ would allow extracting the discrete logarithm via $g^{s_1 - s_2} = (g^\alpha \cdot X^\beta)^{h_1 - h_2}$ where $(\alpha, \beta) = t^*$, thus establishing the EUF-CMA-TK security of $\mathsf{SchnorrSigKP}$ with SchnorrTS.

:::

:::{admonition} Solution
:class: dropdown, solution

We adapt the standard Schnorr security proof by modifying the games to handle tweaking and key prefixing.

**Key modifications:**

1. **Hash function includes public key:** The hash is now computed as $c = \mathsf{H}(R, \tilde{X}, m)$ where $\tilde{X}$ is the tweaked public key.

2. **Signing oracle with tweaking:**
   ```
   Sign(m, t):
     x_tilde := TwSK(x, t)
     X_tilde := TwPK(X, t)
     s ← Z_p
     c ← Z_p
     R := g^s · X_tilde^(-c)
     assert T[R, X_tilde, m] = ⊥
     T[R, X_tilde, m] := c
     Q := Q ∪ {(m, t)}
     return (R, s)
   ```

3. **Collision detection:** The reduction tracks pairs $(t, \tilde{X})$ to detect if different tweaks produce the same tweaked public key.

**Proof that $g^{s^*} = R^* \cdot \mathsf{TwPK}(X, t^*)^{\mathsf{H}(R^*, \mathsf{TwPK}(X, t^*), m^*)}$:**

When $\mathcal{C}$ succeeds, we have $g^{s^*} = R^* \cdot \tilde{X}^{*c^*}$ where $\tilde{X}^* = \mathsf{TwPK}(X, t^*)$ and $c^* = \mathsf{H}_{\mathcal{C}}(R^*, \tilde{X}^*, m^*)$.

We claim that $\mathsf{H}_{\mathcal{C}}(R^*, \tilde{X}^*, m^*) = \mathsf{H}(R^*, \tilde{X}^*, m^*)$.

Assume for contradiction that $\mathsf{H}_{\mathcal{C}}(R^*, \tilde{X}^*, m^*) \neq \mathsf{H}(R^*, \tilde{X}^*, m^*)$.
This means $T[R^*, \tilde{X}^*, m^*]$ was assigned during a signing query.
But whenever the signing oracle sets $T[R, \tilde{X}, m] = c$ for some $(m, t)$ with $\tilde{X} = \mathsf{TwPK}(X, t)$, it also adds $(m, t)$ to $\mathcal{Q}$.

Since $T[R^*, \tilde{X}^*, m^*]$ was set, there must exist some signing query where the key for $T$ was $(R^*, \tilde{X}^*, m^*)$.
This means $R = R^*$, $\tilde{X} = \tilde{X}^*$, and $m = m^*$ for that query.

From $\tilde{X} = \tilde{X}^*$ and $\tilde{X} = \mathsf{TwPK}(X, t)$, $\tilde{X}^* = \mathsf{TwPK}(X, t^*)$, we have $\mathsf{TwPK}(X, t) = \mathsf{TwPK}(X, t^*)$.

If $t \neq t^*$, then $\mathcal{C}$ would have detected this collision and directly extracted the discrete logarithm:
- Let $t = (\alpha, \beta)$ and $t^* = (\alpha^*, \beta^*)$
- Then $g^\alpha \cdot X^\beta = g^{\alpha^*} \cdot X^{\beta^*}$
- This implies $g^\alpha \cdot g^{\beta x} = g^{\alpha^*} \cdot g^{\beta^* x}$
- So $(\beta - \beta^*)x = \alpha^* - \alpha$
- If $\beta = \beta^*$, then $\alpha = \alpha^*$, contradicting $t \neq t^*$
- Thus $\beta \neq \beta^*$ and we can extract $x = (\alpha^* - \alpha)/(\beta - \beta^*) \bmod p$

Otherwise $t = t^*$, so $(m^*, t^*) = (m, t) \in \mathcal{Q}$.
However, $\mathcal{C}$ asserts that $(m^*, t^*) \notin \mathcal{Q}$, which is a contradiction.

Thus $c^* = \mathsf{H}(R^*, \tilde{X}^*, m^*)$.

The bound $|\mathsf{Adv}_{\mathcal{C}, \mathsf{InpGen}}^{\text{Single}} - \mathsf{Adv}_{\mathcal{A}, \mathsf{SchnorrSigKP}, \mathsf{SchnorrTS}}^{\text{EUF-CMA-TK}}| \leq \frac{q_s(q_s + q_h)}{2^{\lambda-1}}$ follows from the same analysis as in the standard lemma, where the error term accounts for the probability of collisions in the programmed oracle.

:::

:::{admonition} Exercise 7 (Optional)
:class: exercise
:label: ex-musig2-tweaking

The security model of the MuSig2 multisignature scheme does not take key tweaking into account.
However, the BIP MuSig2 specification allows for tweaking individual keys (see "Signing with Tweaked Individual Keys").
What problem occurs when using tweaked keys with "vanilla" MuSig2 and how was it fixed?

:::

:::{admonition} Exercise 8 (Optional)
:class: exercise
:label: ex-rerandomizable-keys

Read about signatures with re-randomizable keys in Section 3 of Fleischhacker et al. (PKC 2016), which presents a notion closely related to key tweaking.
Compare their security model with the EUF-CMA-TK model discussed in this section and discuss the practical impact.
How can they prove security for Schnorr signatures with re-randomized keys without requiring public key prefixing?

:::